# Tutorial & Widget Experiments

> This notebook is primarily being used to experiment with how to use the `pymbe` widgets.
>
> It should be fairly "clean" until the `Scratch Pad` section.
>

In the future, this notebook will be cleaned up and renamed "Tutorial".

## 1. Import `pymbe` and create a new user interface

In [ ]:
import pymbe.api as pm

In [ ]:
pm.UI.new(host_url="http://sysml2-sst.intercax.com")

## 2. Use the widget
...or automatically load the `Kerbal` model using the cell below by:

1. Grabbing the individual widgets

In [ ]:
ui, *_ = _.children
client, tree, inspector, lpg = ui.children

2. load a small model

In [ ]:
client.project_selector.value = client.project_selector.options["12b-Allocation"]
client._download_elements()

...and, for example, select some edge types to filter down the diagram

In [ ]:
EDGE_TYPES_TO_SELECT =  ("Superclassing", "FeatureTyping", "FeatureMembership")
lpg.edge_type_selector.value = [
    edges
    for key, edges in lpg.edge_type_selector.options.items()
    if any(
        key.startswith(edge_type)
        for edge_type in EDGE_TYPES_TO_SELECT
    )
]
lpg._update_diagram_graph()

# Interpretation

> This is where we are refactoring the $M_0$ interpretation...

In [ ]:
def safe_get(data: dict, *keys, default=None):
    for key in keys:
        if key not in data:
            return default
        data = data[key]
    return data

safe_get(lpg.elements_by_id, "cb4386fe-67d9-4743-995f-5d4a4162e26e", "owner", "@id")

In [ ]:
UNSPECIFIED = object()

class SafeDict(dict):

    def __init__(self, *args, default=None, **kwargs):
        self.__default__ = default or {}
        super().__init__(*args, **kwargs)
        for key, value in self.items():
            self[key] = SafeDict.__convert(value)

    def __setitem__(self, k, v):
        dict.__setitem__(self, k, SafeDict.__convert(v))

    def get(self, item, default=None):
        return super().get(item, default) or self.__default__

    @staticmethod
    def __convert(o):
        """
        Recursively convert `dict` objects in `dict`, `list`, `set`, and
        `tuple` objects to `attrdict` objects.
        """
        if isinstance(o, dict):
            o = SafeDict(o)
        elif isinstance(o, list):
            o = list(SafeDict.__convert(v) for v in o)
        elif isinstance(o, set):
            o = set(SafeDict.__convert(v) for v in o)
        elif isinstance(o, tuple):
            o = tuple(SafeDict.__convert(v) for v in o)
        return o


a = SafeDict(dict(agh=1, bum=2, bar=3, baz=dict(bar=2,baz=dict(bar=3,baz=4))))
a["as"] = "if"
a.get("as"), type(a.get("baz")), a.get("baz")

In [ ]:
from pymbe.interpretation import *

def roll_up_upper_multiplicities(lpg):
    banded_featuring_graph = make_banded_featuring_graph(lpg)

    banded_roots = [
        banded_featuring_graph.nodes[node]
        for node in banded_featuring_graph
        if banded_featuring_graph.out_degree(node) == 0
    ]
    
    for part_usage in 

elements = lpg.elements_by_id

features = [
    id_
    for id_, data in elements.items()
    if data["@type"] == "Feature"
]

for feature in features:
    upper_multiplicity = get_feature_upper_multiplicity(elements, feature)
    if upper_multiplicity:
        print(elements[feature]["qualifiedName"], upper_multiplicity)

# Troubleshooting

## Fix linking issue with diagram element selector

In [ ]:
ui, *_ = _.children

In [ ]:
client, tree, explorer, lpg = ui.children

In [ ]:
tree.selected, explorer.selected, lpg.selected

In [ ]:
lpg.diagram.elk_app.diagram.selected

In [ ]:
self = lpg.diagram

_, hierarchy = self.elk_transformer.source

self.parts[self.selected[0]], self.elk_app.selected[0]

for selected_id in self.selected:
    node_selected = self.parts.get(selected_id)
    print(node_selected.id)

In [ ]:
node_selected.children[0] == self.elk_app.selected[0].node

# Scratch Pad
> **WARNING**: Anything below this point is just for experimentation purposes

## RDF Experiments

In [ ]:
def process_edge(source, target, edge_type, data=None):
    if edge_type in ("FeatureTyping", "FeatureMembership"):
        source, target = target, source
    return [source, target, data]

graph = nx.DiGraph()
graph.add_edges_from([
    process_edge(source, target, edge_type, data)
    for (source, target, edge_type), data in dict(client.lpg.graph.edges).items()
    if edge_type in ("Superclassing", "FeatureTyping", "FeatureMembership")
])
graph

In [ ]:
diagram.graph = graph

In [ ]:
diagram = client.lpg.make_diagram(
    graph=client.lpg.subgraph(edge_types=("Superclassing", "FeatureTyping^-1", "FeatureMembership^-1"))
)
elk_app, *_ = diagram.children
diagram

In [ ]:
# Select the root node in the diagram...
# ... or you can manually select one yourself
elk_app.selected = "5260380b-6fda-43cc-993f-5df58868edbb",

In [ ]:
first_element_selected, *_ = elk_app.selected
client.elements_by_id[first_element_selected]

# Parse JSON-LD into RDF

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

result = client.rdf.graph
# result = g.parse(url, format='turtle')

G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')

In [ ]:
ax = plt.figure(figsize=(50,30)).gca();
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
nx.draw(G, with_labels=True, ax=ax)

# TODOs
1. Finish fleshing out the process in the `Kerbal Model.ipynb`
2. Modify the subgraph generator so it can take the value from the `Type Selector` directly
3. Improve the ipyelk diagram widget (may need to make improvements to `ipyelk`)
   * Add arrows
   * Add compartments
   * Fix layout
   * Add widget to see node details
4. Finalize the RDF formulation